Intro to cuDF
=======================

Welcome to first cuDF tutorial notebook! This is a short introduction to cuDF, partly modeled after 10 Minutes to Pandas, geared primarily for new users. cuDF is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating data. While we'll only cover some of the cuDF functionality, but at the end of this tutorial we hope you'll feel confident creating and analyzing GPU DataFrames.

We'll start by introducing the pandas library, and quickly move on cuDF.

In [ ]:
import os
import numpy as np
import math
np.random.seed(12)

<a id="pandas"></a>
## Pandas

Data scientists typically work with two types of data: unstructured and structured. Unstructured data often comes in the form of text, images, or videos. Structured data - as the name suggests - comes in a structured form, often represented by a table or CSV. We'll focus the majority of these tutorials on working with structured data.

There exist many tools in the Python ecosystem for working with structured, tabular data but few are as widely used as Pandas. Pandas represents data in a table and allows a data scientist to manipulate the data to perform a number of useful operations such as filtering, transforming, aggregating, merging, visualizing and many more. 

For more information on Pandas, check out the excellent documentation: http://pandas.pydata.org/pandas-docs/stable/

Below we show how to create a Pandas DataFrame, an internal object for representing tabular data.

In [ ]:
import pandas as pd; print('Pandas Version:', pd.__version__)


# here we create a Pandas DataFrame with
# two columns named "key" and "value"
df = pd.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
print(df)

We can perform many operations on this data. For example, let's say we wanted to sum all values in the in the `value` column. We could accomplish this using the following syntax:

In [ ]:
aggregation = df['value'].sum()
print(aggregation)

<a id="cudf"></a>
## cuDF

Pandas is fantastic for working with small datasets that fit into your system's memory and workflows that are not computationally intense. However, datasets are growing larger and data scientists are working with increasingly complex workloads - the need for accelerated computing is increasing rapidly.

cuDF is a package within the RAPIDS ecosystem that allows data scientists to easily migrate their existing Pandas workflows from CPU to GPU, where computations can leverage the immense parallelization that GPUs provide.

Below, we show how to create a cuDF DataFrame.

In [ ]:
import cudf; print('cuDF Version:', cudf.__version__)


# here we create a cuDF DataFrame with
# two columns named "key" and "value"
df = cudf.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
print(df)

As before, we can take this cuDF DataFrame and perform a `sum` operation over the `value` column. The key difference is that any operations we perform using cuDF use the GPU instead of the CPU.

In [ ]:
aggregation = df['value'].sum()
print(aggregation)

Note how the syntax for both creating and manipulating a cuDF DataFrame is identical to the syntax necessary to create and manipulate Pandas DataFrames; the cuDF API is based on the Pandas API. This design choice minimizes the cognitive burden of switching from a CPU based workflow to a GPU based workflow and allows data scientists to focus on solving problems while benefitting from the speed of a GPU!

# DataFrame Basics with cuDF

In the following tutorial, you'll get a chance to familiarize yourself with cuDF. For those of you with experience using pandas, this should look nearly identical.

Along the way you'll notice small exercises. These exercises are designed to help you get a feel for writing the code yourself, but if you get stuck, you can take a look at the solutions.

Portions of this were borrowed from the 10 Minutes to cuDF guide.

Object Creation
---------------

Creating a `cudf.Series`.

In [ ]:
s = cudf.Series([1,2,3,None,4])
print(s)

Creating a `cudf.DataFrame` by specifying values for each column.

In [ ]:
df = cudf.DataFrame([('a', list(range(20))),
('b', list(reversed(range(20)))),
('c', list(range(20)))])
print(df)

Creating a `cudf.DataFrame` from a `pd.Dataframe`.

In [ ]:
pdf = pd.DataFrame({'a': [0, 1, 2, 3],'b': [0.1, 0.2, None, 0.3]})
gdf = cudf.DataFrame.from_pandas(pdf)
print(gdf)

Viewing Data
-------------

Viewing the top rows of a GPU dataframe.

In [ ]:
print(df.head(2))

Sorting by values.

In [ ]:
print(df.sort_values(by='b'))

Selection
------------

## Getting

Selecting a single column, which initially yields a `cudf.Series` (equivalent to `df.a`).

In [ ]:
print(df['a'])

## Selection by Label

Selecting rows from index 2 to index 5 from columns `a` and `b`.

In [ ]:
print(df.loc[2:5, ['a', 'b']])

## Selection by Position

Selecting via integers and integer slices, like numpy/pandas.

In [ ]:
print(df.iloc[0])

In [ ]:
print(df.iloc[0:3, 0:2])

You can also select elements of a `DataFrame` or `Series` with direct index access.

In [ ]:
print(df[3:5])

In [ ]:
print(s[3:5])

## Exercise 1

Try to select only the rows at index `4` and `9` from `df`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.iloc[[4,9]])
   </pre>
</details>

## Boolean Indexing

Selecting rows in a `DataFrame` or `Series` by direct Boolean indexing.

In [ ]:
print(df[df.b > 15])

Selecting values from a `DataFrame` where a Boolean condition is met, via the `query` API.

In [ ]:
print(df.query("b == 3"))  

In [ ]:
val = 3
df.query("b == @val")

You can also pass local variables to cuDF queries, via the `local_dict` keyword or `@` operator.

In [ ]:
cudf_comparator = 3
print(df.query("b == @cudf_comparator"))

Supported logical operators include `>`, `<`, `>=`, `<=`, `==`, and `!=`.

## Exercise 2

Try to select only the rows from `df` where the value in column `b` is greater than the value in column `c` + 6.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.query("b > c + 6"))
   </pre>
</details>

Missing Data
------------

Missing data can be replaced by using the `fillna` method.

In [ ]:
print(s.fillna(999))

Operations
------------

## Stats

Calculating descriptive statistics for a `Series`.

In [ ]:
s = cudf.Series(np.arange(10)).astype(np.float32)

In [ ]:
print(s.mean(), s.var())

In [ ]:
print(df.describe())

## Applymap

Applying functions to a `Series`.

In [ ]:
def add_ten(num):
    return num + 10

print(s.applymap(add_ten))

We can also apply more complicated functions, such as this one.

In [ ]:
def complex_math_transform(num):
    return math.cos(num) * 3 / 9

print(s.applymap(complex_math_transform))

## String Methods

Like pandas, cuDF provides string processing methods in the `str` attribute of `Series`. Full documentation of string methods is a work in progress. Please see the cuDF and [nvStrings API](https://docs.rapids.ai/api/nvstrings/nightly/) documentation for more information.

In [ ]:
s = cudf.Series(['A', 'B', 'C', 'Aaba', 'Baca', None, 'CABA', 'dog', 'cat'])
print(s.str.lower())

## Exercise 3

Try to convert all the strings to uppercase. Take a look at the nvStrings API documentation linked above if you need some help.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(s.str.upper())
   </pre>
</details>

## Concat

Concatenating `Series` and `DataFrames` row-wise.

In [ ]:
s = cudf.Series([1, 2, 3, None, 5])
print(cudf.concat([s, s]))

## Append

Appending values from another `Series` or array-like object.

In [ ]:
print(s.append(s))

## Join

Performing SQL style merges. Note that the dataframe order is not maintained, but may be restored post-merge by sorting by the index.

In [ ]:
df_a = cudf.DataFrame()
df_a['key'] = ['a', 'b', 'c', 'd', 'e']
df_a['vals_a'] = [float(i + 10) for i in range(5)]

df_b = cudf.DataFrame()
df_b['key'] = ['a', 'c', 'e']
df_b['vals_b'] = [float(i+100) for i in range(3)]

merged = df_a.merge(df_b, on=['key'], how='left')
print(merged)

## Exercise 4

Using the DataFrames we created above, try to do an `inner` join using `merge`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df_a.merge(df_b, on=['key'], how='inner'))
   </pre>
</details>

## Grouping

Like pandas, cuDF supports the [Split-Apply-Combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) groupby paradigm.

In [ ]:
df['agg_col1'] = [1 if x % 2 == 0 else 0 for x in range(len(df))]
df['agg_col2'] = [1 if x % 3 == 0 else 0 for x in range(len(df))]

Grouping and then applying the `sum` function to the grouped data.

In [ ]:
print(df.groupby('agg_col1').sum())

Grouping and applying statistical functions to specific columns, using `agg`.

In [ ]:
print(df.groupby('agg_col1').agg({'a':'max', 'b':'mean', 'c':'sum'}))

## Exercise 5

We can also group by multiple columns at once, which we call grouping hierarchically. Try to group `df` by `agg_col1` and `agg_col2` and calculate the mean of column `a` and minimum of column `b`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.groupby(['agg_col1', 'agg_col2']).agg({'a':'mean', 'b':'min'}).to_pandas())
   </pre>
</details>

Time Series
------------


`DataFrames` supports `datetime` typed columns, which allow users to interact with and filter data based on specific timestamps.

In [ ]:
date_df = cudf.DataFrame()
date_df['date'] = pd.date_range('11/20/2018', periods=72, freq='D')
date_df['value'] = np.random.sample(len(date_df))
print(date_df.head())

## Exercise 6

Try to use `query` to filter `date_df` to only those row with a date before `2018-11-23`. This is a bit trickier than the prior exercises. As a hint, you'll want to explore the `to_datetime` function from the `pandas` library.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>
    search_date = pd.to_datetime('2018-11-23')
    print(date_df.query("date &lt= @search_date"))
            </br>
   </pre>
</details>

You can also interact with datetime columns to extract things like the day, hour, minute, and more.

In [ ]:
date_df['minute'] = date_df.date.dt.minute
print(date_df.head())

Converting Data Representation
--------------------------------

## Pandas

Converting a cuDF `DataFrame` to a pandas `DataFrame`.

In [ ]:
print(df.head().to_pandas())

## Numpy

Converting a cuDF `DataFrame` to a numpy `ndarray`.

In [ ]:
print(df.as_matrix())

Converting a cuDF `Series` to a numpy `ndarray`.

In [ ]:
print(df['a'].to_array())

Getting Data In/Out
------------------------


## CSV

Writing to a CSV file, by first sending data to a pandas `Dataframe` on the host.

In [ ]:
if not os.path.exists('example_output'):
    os.mkdir('example_output')
    
df.to_pandas().to_csv('example_output/foo.csv', index=False)

Reading from a csv file.

In [ ]:
df = cudf.read_csv('example_output/foo.csv')
print(df)

That's it! You've got the basics of cuDF down! Let's talk a little bit about the computational performance of cuDF and GPUs.

# Performace

One of the primary reasons to use cuDF over pandas is performance. For some workflows, the GPU can be **much** faster than the CPU. Let's illustrate this by starting with a small example: creating a DataFrame and calculating the sum of a column.

In [ ]:
a = np.random.rand(10000000) # 10 million values

In [ ]:
pdf = pd.DataFrame()
cdf = cudf.DataFrame()

In [ ]:
%time pdf['a'] = a
%time cdf['a'] = a

In [ ]:
%%timeit
pdf['a'].sum()

In [ ]:
%%timeit
cdf['a'].sum()

Pretty cool! This is a pretty small example, though.

### A More Realistic Example: Sensor Data Analytics

To get a more realistic sense of how powerful cuDF and GPUs can be, let's imagine you had a fleet of sensors that collect data every millisecond. These sensors could be measuring pressure, temperature, or something else entirely.

Let's imagine we want to analyze one day's worth of sensor data. We'll assign random values for the sensor `value` to use for this example. We'll start by creating the data, and generating some datetime related features like we learned about in the above tutorial.

In [ ]:
%%time

date_df = pd.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute

print(date_df.shape)
date_df.head()

Just creating the data takes a while! Let's do our analysis. From our sensor data, we want to get the maximum sensor value for each minute. Since we don't want to combine values in the same minute of different hours, we'll need to do a hierarchical groupby.

In [ ]:
%time results = date_df.groupby(['hour', 'minute']).agg({'value':'max'})
results.head()

This is fairly slow! Imagine if we had a fleet of sensors. Time would become a serious concern.

Let's try this in cuDF now, using the GPU DataFrame. We'll run the same code as above.

In [ ]:
%%time

date_df = cudf.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute
date_df['second'] = date_df.date.dt.second

print(date_df.shape)
print(date_df.head())

In [ ]:
%time results = date_df.groupby(['hour', 'minute', 'second']).agg({'value':'max'})
results.head()

While the results may vary slightly, it should be clear that GPU acceleration can make a significant difference. We can get much faster results with the same code!

## Exercise 7

Play around with some more pandas and cuDF operations and compare the performance between them. What operations can you find that gives the highest performance ratio? 

You can start a cell with `%%time` to time the cell, or with `%%timeit`, which runs the cell multiple times and gives an average. `%%timeit` gives a more accurate benchmark but takes longer to run.